In [252]:
import numpy as np
import pandas as pd
from pathlib import Path
import datetime
import warnings
import numpy as np
import json
import copy
import urllib.request
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 500
pd.options.display.max_rows = 1000

In [243]:
# import data
dir = str(Path().resolve())
air = pd.read_csv(dir + "/../processing/data/airbnb/counts.csv")
apts = pd.read_csv(dir + "/../processing/data/apts/counts.csv")
with urllib.request.urlopen("http://bostonopendata-boston.opendata.arcgis.com/datasets/53ea466a189b4f43b3dfb7b38fa7f3b6_1.geojson") as url:
    zips = json.loads(url.read().decode())

In [244]:
air = air.drop('counts', axis=1)
apts = apts.drop('counts', axis=1)

In [245]:
air['density'] = air['density'] / air.density.max()
apts['density'] = apts['density'] / apts.density.max()

In [246]:
apts['zipcode'] = "0" + apts['zipcode'].apply(str)

In [247]:
zips = zips['features']

In [248]:
for z in zips:
    z['id'] = z['properties']['ZIP5']
    z.pop('properties')
    y = z.copy()
    z['source'] = {'type:': 'geojson', 'data':y}
    z.pop('geometry')
    z['source']['data'].pop('id')
    z['type'] = 'fill'

In [249]:
zips[2]

{'type': 'fill',
 'id': '02110',
 'source': {'type:': 'geojson',
  'data': {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-71.05109058896998, 42.36418367507441],
      [-71.0510885894623, 42.36407367476054],
      [-71.05111558843308, 42.36398767512105],
      [-71.0511955883311, 42.363728674930265],
      [-71.05135058904166, 42.36347867493753],
      [-71.05141458934484, 42.36339467493657],
      [-71.05164758926375, 42.36311867422857],
      [-71.0517785888345, 42.36295067467764],
      [-71.05189858886018, 42.36281367475747],
      [-71.05106358900065, 42.36225667407527],
      [-71.05110158877561, 42.36218267451762],
      [-71.05120158917359, 42.36207467460202],
      [-71.05137858870212, 42.361948674075535],
      [-71.05195658844426, 42.36167667445881],
      [-71.05226458838904, 42.36190467425306],
      [-71.05250358913817, 42.36209067445736],
      [-71.05332658925471, 42.36119567450176],
      [-71.05343158965823, 42.36111267402365],
      [-71

In [253]:
apt_geo = copy.deepcopy(zips)
air_geo = copy.deepcopy(zips)

In [254]:
for y in apt_geo:
    y['layout'] = {}
    if y['id'] in apts.zipcode.unique():
        y['paint'] = {'fill-color': "#FF0000", 
                      'fill-opacity': apts.density.iloc[apts.zipcode[apts.zipcode==y['id']].index[0]]}
    else:
        y['paint'] = {'fill-color': "FF0000", 'fill-opacity': 0}

In [255]:
for z in air_geo:
    y['layout'] = {}
    if z['id'] in air.zipcode.unique():
        z['paint'] = {'fill-color': "#FF0000", 
                      'fill-opacity': air.density.iloc[air.zipcode[air.zipcode==z['id']].index[0]]}
    else:
        z['paint'] = {'fill-color': "FF0000", 'fill-opacity': 0}

In [256]:
air_geo[0]

{'type': 'fill',
 'id': '02134',
 'source': {'type:': 'geojson',
  'data': {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-71.12340461235522, 42.36420867214283],
      [-71.12344661248451, 42.36440467252391],
      [-71.12345161261547, 42.36449067279288],
      [-71.12337061278323, 42.3647236726063],
      [-71.12334661233186, 42.36480567291731],
      [-71.12330861185814, 42.364933672646536],
      [-71.12323261201861, 42.365187672926496],
      [-71.1233426131028, 42.36522467251061],
      [-71.12340361270336, 42.365266672635734],
      [-71.12342261321453, 42.365311672893014],
      [-71.12338861271265, 42.3655176724321],
      [-71.12372661280114, 42.365609672520996],
      [-71.1241336131843, 42.365735672841225],
      [-71.12477161271094, 42.36599367327273],
      [-71.1251246128844, 42.36615367339508],
      [-71.1255736131498, 42.36640767338003],
      [-71.1252136139545, 42.36678767298522],
      [-71.12498461255515, 42.36701267282802],
      [-71

In [257]:
apt_geo[0]

{'type': 'fill',
 'id': '02134',
 'source': {'type:': 'geojson',
  'data': {'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-71.12340461235522, 42.36420867214283],
      [-71.12344661248451, 42.36440467252391],
      [-71.12345161261547, 42.36449067279288],
      [-71.12337061278323, 42.3647236726063],
      [-71.12334661233186, 42.36480567291731],
      [-71.12330861185814, 42.364933672646536],
      [-71.12323261201861, 42.365187672926496],
      [-71.1233426131028, 42.36522467251061],
      [-71.12340361270336, 42.365266672635734],
      [-71.12342261321453, 42.365311672893014],
      [-71.12338861271265, 42.3655176724321],
      [-71.12372661280114, 42.365609672520996],
      [-71.1241336131843, 42.365735672841225],
      [-71.12477161271094, 42.36599367327273],
      [-71.1251246128844, 42.36615367339508],
      [-71.1255736131498, 42.36640767338003],
      [-71.1252136139545, 42.36678767298522],
      [-71.12498461255515, 42.36701267282802],
      [-71

In [259]:
with open('apts.json', 'w') as outfile:  
    json.dump(apt_geo, outfile)

In [260]:
with open('airs.json', 'w') as outfile:  
    json.dump(apt_geo, outfile)